## Noise model walkthrough

This notebook will give a visual guide through the `SimpleNoiseModel` utility and discuss some of the reasoning behind the implementation.

### Start

Lets define a simple circuit with a minimal set of gates that lets us see how the noise model modifies the circuit.

In [1]:
import cirq
import numpy as np
from simple_noise_model import SimpleNoiseModel

In [2]:
q0, q1, q2 = cirq.LineQubit.range(3)
qubits = [q0, q1, q2]
ops = [
    cirq.rx(np.pi/3).on(q0),
    cirq.H(q2),
    (cirq.ISWAP ** 0.5).on(q1, q2),
]
circuit = cirq.Circuit(*ops)
measurements = [    
    cirq.measure(q0, key='q0'),
    cirq.measure(q1, key='q1'),
    cirq.measure(q2, key='q2'),
]
circuit += cirq.Circuit(*measurements)
print(circuit)

0: ───Rx(0.333π)───────────────M('q0')───

1: ────────────────iSwap───────M('q1')───
                   │
2: ───H────────────iSwap^0.5───M('q2')───


### Now add T1 based on hardware gate durations

In [3]:
# Come up with some T1 values and construct a mapping between qubits and T1's
t1_lst = [100, 121, 87] # T1 decay parameters always in us
t1_noise_only = SimpleNoiseModel.from_fixed_values()

# Note that this dictionary has tuple-typed keys for compatibility with two-qubit gates later on
qubit_keys = [tuple([q]) for q in qubits]
t1_map = dict(zip(qubit_keys, t1_lst))
t1_noise_only.T1_map = t1_map

t1_circuit = circuit.with_noise(t1_noise_only)
print(t1_circuit)

0: ───AD(0.00025)────Rx(0.333π)───AD(0.00032)────────────────M('q0')───

1: ───AD(0.000207)────────────────AD(0.000264)───iSwap───────M('q1')───
                                                 │
2: ───AD(0.000287)───H────────────AD(0.000368)───iSwap^0.5───M('q2')───


Before every moment of operations there is now an amplitude damping channel that simultes T1 decay corresponding to the length of the longest gate in that moment and the T1 on each gate. The first column of AD's persists for a time that is maximum gate execution time between $H$ and $R_x$, the second column of AD's corresponds for a time that is the gate execution time of $\sqrt{\text{iSWAP}}$.

### Now add T2 based on hardware gate durations

For more complicated parameterized models its useful to initialize from a master dictionary containing all relevant calibration data.

In [4]:
# Come up with some T2 values
t2_lst = [66, 36, 121] # T2 decay parameters always in us
t2_map = dict(zip(qubit_keys, t2_lst))

calibration_dct = {
    "single_qubit_t1": t1_map,
    "single_qubit_t2": t2_map,
}
 
t1_t2_noise = SimpleNoiseModel.from_calibration_data(calibration_dct)

t1_t2_circuit = circuit.with_noise(t1_t2_noise)
print(t1_t2_circuit)

0: ───AD(0.00025)────PD(0.000379)───Rx(0.333π)───AD(0.00032)────PD(0.000485)───────────────M('q0')───

1: ───AD(0.000207)───PD(0.000695)────────────────AD(0.000264)───PD(0.000889)───iSwap───────M('q1')───
                                                                               │
2: ───AD(0.000287)───PD(0.000207)───H────────────AD(0.000368)───PD(0.000265)───iSwap^0.5───M('q2')───


Notice how now every Amplitude Damping channel (AD) is paired up with a Phase Damping channel (PD) corresponding to T2 decay for the same moment.

### Now add 1-qubit gate fidelity

Now we add single qubit gate infidelities. This will typically "clash" with T1/T2 decay since gate fidelity experiments absorb decay into the measured fidelity, so we carefully opt out of applying T1/T2 noise to the gates for which single qubit gate fidelity was applied.


In [5]:
# Come up with some single qubit gate fidelities
fid_1q_lst = [.003, .010, .009] # these are gate _infidelities_; there is a legacy naming system in place...
fid_1q_map = dict(zip(qubit_keys, fid_1q_lst))

calibration_dct.update({"fid_1q": fid_1q_map})
 
t1_t2_1qrb_noise = SimpleNoiseModel.from_calibration_data(calibration_dct)

t1_t2_1qrb_circuit = circuit.with_noise(t1_t2_1qrb_noise)
print(t1_t2_1qrb_circuit)

0: ───D(0.00225)────────────────────Rx(0.333π)───AD(0.00032)────PD(0.000485)───────────────M('q0')───

1: ───AD(0.000207)───PD(0.000695)────────────────AD(0.000264)───PD(0.000889)───iSwap───────M('q1')───
                                                                               │
2: ───D(0.00675)────────────────────H────────────AD(0.000368)───PD(0.000265)───iSwap^0.5───M('q2')───


Notice how in the first moment, qubits 0 and 2 now _only_ have depolarizing noise (corresponding to the gate infidelity parameters) while qubit 1 still decays via $T_1$, $T_2$ based on the longest gate duration in that moment. The $\sqrt{iSWAP}$ is left untouched because we did not provide 2-qubit gate infidelity, and the measurements are untouched because by default they will _not_ be time-decayed since this is a poor model for bitflip errors.

### Now add 2-qubit gate fidelity

For two-qubit decoherence, we will assume two sources: unitary control error, and decoherent error. This is an extremely **speculative** noise model for two-qubit error. I **highly recommend** that we further explore whether this is something we want to include at all.

In [6]:
sqrtISWAP_12_tot_fid = 0.07 # 2q gate total error parameter for qubits 1 and 2
sqrtISWAP_12_pure_fid = 0.03 # 2q gate decoherent error parameter for qubits 1 and 2

# Now the qubit map has to be constructed between pairs of qubits
fid_2q_tot_map = {(q1, q2): sqrtISWAP_12_tot_fid}
fid_2q_pure_map = {(q1, q2): sqrtISWAP_12_pure_fid}

calibration_dct.update({"fid_2q_tot": fid_2q_tot_map})
calibration_dct.update({"fid_2q_pure": fid_2q_pure_map})

t1_t2_1qrb_2q_noise = SimpleNoiseModel.from_calibration_data(calibration_dct)

t1_t2_1qrb_2q_circuit = circuit.with_noise(t1_t2_1qrb_2q_noise)
print(t1_t2_1qrb_2q_circuit)


0: ───D(0.00225)────────────────────Rx(0.333π)───AD(0.00032)────PD(0.000485)───────────────M('q0')───

1: ───AD(0.000207)───PD(0.000695)────────────────2Q-S-D(0.03)───[X]────────────iSwap───────M('q1')───
                                                 │              │              │
2: ───D(0.00675)────────────────────H────────────2Q-S-D(0.03)───[X]^-0.0255────iSwap^0.5───M('q2')───


We now see two new features
 - `2Q-S-D`: two-qubit symmetric depolarizing channel parameterized by `fid_2q_pure`, the "purity error" in the gate
 - Some other two-local gate of the form $\exp(i \theta \sigma_i \otimes \sigma_j)$ where $\theta$ is determined by the `fid_2q_tot - fid_2q_pure`.
 
And as with single qubit gate fidelities, we have removed the `AD` and `PD` channels that were previously applied to $\sqrt{iSWAP}$ as those decays should be accounted for in whatever two qubit gate infidelities we provide.

### Now add measurement error

Finally, we add simple asymmetric measurement error channels. These are typically more appropriately modelled by a $2^n \times 2^n$ "response matrix" that allows for correlated readout error between measurements. In that case, you should disable measurement error in the noise model and use a response matrix, but this is an _okay_ substitute if you don't have the resources to do full readout error spectroscopy.

As a general rule, $p_1 > p_0$ for any given qubit.

In [7]:
p0_errors = [0.03, 0.02, 0.05] # bitwise probability of "0" flipping to "1"
p0_map = dict(zip(qubit_keys, p0_errors))

p1_errors = [0.07, 0.12, 0.09] # bitwise probability of "1" flipping to "0" 
p1_map = dict(zip(qubit_keys, p1_errors))

calibration_dct.update({"p0_readout_error": p0_map})
calibration_dct.update({"p1_readout_error": p1_map})

full_noise_model = SimpleNoiseModel.from_calibration_data(calibration_dct)
full_noise_circuit = circuit.with_noise(full_noise_model)

print(full_noise_circuit)


0: ───D(0.00225)────────────────────Rx(0.333π)───AD(0.00032)────PD(0.000485)───────────────ABF(0.03,0.07)───M('q0')───

1: ───AD(0.000207)───PD(0.000695)────────────────2Q-S-D(0.03)───[Z]────────────iSwap───────ABF(0.02,0.12)───M('q1')───
                                                 │              │              │
2: ───D(0.00675)────────────────────H────────────2Q-S-D(0.03)───[X]^-0.0255────iSwap^0.5───ABF(0.05,0.09)───M('q2')───


Now we see `ABF` standing for asymmetric bitflip, with the parameters that we provided. All this is is a matrix of the form
$$
\begin{pmatrix}
    1 - p_0 & p_1 \\
    p_0 & 1 - p_1
\end{pmatrix}
$$
applied to each qubit, which gets decomposed into some Kraus operators so that its a proper CPTP map.

#### This concludes the noise model